I compile a common dataset for DAX activity. Other assets are calculated similarly, only by changing variable names.

In [ ]:
DAX = pd.concat([Index[["Unnamed: 6", "DAX Index"]], RiskFree["Germany RF Index"]], axis=1)

We calculate the DAX investment index. (ROI)

In [ ]:
def index(df, col):
    first_val = df[col].iloc[0]
    index = df[col].div(first_val)
    return index

DAX['Index DAX'] = index(DAX,"DAX Index")

We calculate the monthly return of DAX.

In [ ]:
def Return(df, col):
    Return = df[col].div(df[col].shift())  # divide each value by the previous value
    Return.iloc[0] = 1  # set the first value to 1
    Return = (Return - 1)  # subtract 1 from each value
    return Return

DAX['Return DAX'] = Return(DAX,"Index DAX")

We are calculating the monthly return of Germany RF.
(For DAX and S&P500 assets, we calculated the US RF return)

In [ ]:
def ReturnUS(df, col):
    ReturnUS = df[col].div(df[col].shift()) previous value
    ReturnUS.iloc[0] = 1
    ReturnUS = (ReturnUS - 1)
    return ReturnUS

DAX['Return GERRF'] = ReturnUS(DAX,"Germany RF Index")

Monthly returns of DAX above risk-free investment

In [ ]:
def Return_month(df, col1, col2):
    Return_month = df[col1] - df[col2]  # divide each value by the first value
    return Return_month

DAX['Return Month'] = Return_month(DAX,"Return DAX","Return GERRF")

Plot the price and return changes for the DAX asset.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

axs[0].plot(DAX['Unnamed: 6'], DAX['DAX Index'],linewidth=2)
axs[0].set_ylabel('Akcijos kaina',fontsize=20)
axs[0].set_title('DAX mėnesinis akcijų kainos pokytis',size=20)

axs[1].plot(DAX['Unnamed: 6'], DAX['Return DAX'], linewidth=2)
axs[1].set_ylabel('Grąžos dydis',fontsize=20)
axs[1].set_title('DAX mėnesinis grąžų pokytis',size=20)

plt.show()

Line chart of all returns above risk-free investment

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(15, 10), sharex=True)


plt.plot(DAX['Unnamed: 6'], DAX['Return Month'], linewidth=2, label='DAX')
plt.plot(S_P500['Unnamed: 0'], S_P500['Return Month'], linewidth=2, label='S&P500')
plt.plot(REIT['Unnamed: 2'], REIT['Return Month'], linewidth=2, label='REIT')
plt.ylabel('Grąža', fontsize=20)
plt.title('Grąžos virš NI', size=20)
plt.legend(fontsize=16)

Total for all months

In [ ]:
num_month = S_P500.shape[0] - 1
print('Number of rows:', num_month)

Calculation of average returns above Risk-Free Investment

In [ ]:
Return_NI_DAX = ((DAX["Index DAX"].iloc[-1] / DAX["Germany RF Index"].iloc[-1])**(12/num_month))-1

We perform a test to verify whether the average statistically differs from 0 for the DAX asset.

In [ ]:
t_stat, p_value = ttest_1samp(DAX['Return Month'], 0)
print("T-statistic value: ", t_stat)
print("P-Value: ", p_value)

We calculate the standard deviation.

In [ ]:
std_dev_DAX = (DAX["Return Month"].std())*np.sqrt(12)

Calculation of the Sharpe ratio.

In [ ]:
Sharpe_Ratio_DAX = Return_NI_DAX / std_dev_DAX

Calculation of skewness and kurtosis.

In [ ]:
DAX_skew = skew(DAX['Return Month'])
DAX_kurt = kurtosis(DAX['Return Month'])

We check whether these indicators significantly differ from a normal distribution.

In [ ]:
DAX_skew_test = skewtest(DAX['Return Month'])
DAX_kurt_test = kurtosistest(DAX['Return Month'])

Calculation of VaR and CVaR.

In [ ]:
VaR999_SP = Return_NI_SP + (std_dev_SP * norm.ppf(0.001))
VaR95_SP = Return_NI_SP + (std_dev_SP * norm.ppf(0.05))
VaR99_SP = Return_NI_SP + (std_dev_SP * norm.ppf(0.01))

VaR999_DAX = Return_NI_DAX + (std_dev_DAX * norm.ppf(0.001))
VaR95_DAX = Return_NI_DAX + (std_dev_DAX * norm.ppf(0.05))
VaR99_DAX = Return_NI_DAX + (std_dev_DAX * norm.ppf(0.01))

z999 = norm.ppf(1-con999)
z95 = norm.ppf(1-con95)
z99 = norm.ppf(1-con99)

CVaR999_DAX = 1 * (Return_NI_DAX - std_dev_DAX * (norm.pdf(z999)/(1-con999)))
CVaR95_DAX = 1 * (Return_NI_DAX - std_dev_DAX * (norm.pdf(z95)/(1-con95)))
CVaR99_DAX = 1 * (Return_NI_DAX - std_dev_DAX * (norm.pdf(z99)/(1-con99)))

Application of the CAPM model.

In [ ]:
Market_Return_DAX = ((DAX["Germany RF Index"].iloc[-1] / DAX["Germany RF Index"].iloc[1])**(12/num_month))-1
covariance_DAX = DAX["DAX return"].cov(DAX["RF return"])
variance_DAX = DAX["RF return"].var()
beta_DAX = covariance_DAX / variance_DAX
print("Beta of DAX:", beta_DAX)
CAPM_DAX = Return_NI_DAX + beta_DAX * (Market_Return_DAX-Return_NI_DAX)
print("CAPM model value of DAX:", CAPM_DAX)

Calculating the logarithms of returns.

In [ ]:
def Log_Return(df, col):
    log_return = np.log(df[col]) - np.log(df[col].shift())
    log_return.iloc[0] = 0
    return log_return

def Return_month_log(df, col1, col2):
    Return_month_log = df[col1] - df[col2]
    return Return_month_log

Checking assumptions for data stationarity.

In [ ]:
result_DAX = adfuller(DAX['Monthly log return'])
print('ADF Statistic of DAX: %f' % result_DAX[0])
print('p-value: %f' % result_DAX[1])
print('Critical Values:')
for key, value in result_DAX[4].items():
print('\t%s: %.3f' % (key, value))

Testing residuals for independence.

In [ ]:
sm.stats.acorr_ljungbox(DAX['Monthly log return'],  lag = [1], return_df=True)

Assumptions about residuals' normal distribution.

In [ ]:
jb_stat_DAX, jb_pval_DAX = jarque_bera(DAX['Monthly log return'])
if jb_pval_DAX < 0.05:
    print("The distribution of DAX returns is significantly non-normal (p < 0.05)", jb_pval_DAX)
else:
    print("The distribution of DAX returns is not significantly non-normal (p >= 0.05)")
print(f"Jarque-Bera test statistic: {jb_stat_DAX:.4f}")

Optimal lag selection for the GARCH model.

In [ ]:
def garch_lag(df, col, max_lag=13):
    best_lag = None
    best_aic = np.inf

    for lag in range(1, max_lag + 1):
        gim = ARCHInMean(df[col], lags=[lag], volatility=GARCH(p=1, q=1), distribution=GeneralizedError(), form='var')
        res = gim.fit(disp='off')
        aic = res.aic
        if aic < best_aic:
            best_lag = lag
            best_aic = aic

    print("Best Lag Structure:", [best_lag])
    print("AIC:", best_aic)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('Best lag for S_P500')
    garch_lag(S_P500, "Monthly log return", max_lag=12)
    print('Best lag for DAX')
    garch_lag(DAX, "Monthly log return", max_lag=12)
    print('Best lag for REIT')
    garch_lag(REIT, "Monthly log return", max_lag=12)

Construction of the GARCH-IN-MEAN model.

In [ ]:
GARCH_DAX = ARCHInMean(DAX['Monthly log return'], lags=[1], volatility=GARCH(p=1, q=1), distribution=GeneralizedError(), form='var')
GARCH_DAX_results = GARCH_DAX.fit(disp='off')

print(GARCH_DAX_results.summary())

Application of rolling regression.

In [ ]:
window_size = 96
rolling_avg_returns = []
rolling_beta_values = []
rolling_std_values = []

for i in range(window_size, len(DAX["Monthly return"]) + 1):
    subset = DAX["Monthly return"].iloc[i-window_size:i]
    subset_complex = subset.apply(lambda x: complex(x, 0))

    abs_subset = np.abs(subset_complex)
    sign_subset = np.sign(subset_complex)

    # average
    avg_return = np.float_power(abs_subset.iloc[-1] / abs_subset.iloc[0], 1/window_size) - 1
    avg_return = avg_return * sign_subset.iloc[0]
    rolling_avg_returns.append(avg_return)
    # beta
    beta_subset = DAX['DAX return'].iloc[i-window_size:i]
    rolling_beta = beta_subset.cov(DAX['RF return']) / DAX['RF return'].var()
    rolling_beta_values.append(rolling_beta)
    # std
    rolling_std = subset.std() * np.sqrt(12)
    rolling_std_values.append(rolling_std)


rolling_beta = pd.Series(rolling_beta_values + [np.nan] * (len(DAX["DAX return"]) - len(rolling_beta_values)))
rolling_avg_returns = pd.Series(rolling_avg_returns + [np.nan] * (len(DAX["Monthly return"]) - len(rolling_avg_returns)))
rolling_std = pd.Series(rolling_std_values + [np.nan] * (len(S_P500["Monthly return"]) - len(rolling_std_values)))

We save the data in a new dataset.

In [ ]:
rolling_df = pd.DataFrame({
    'Date': DAX['Date'].iloc[window_size-1:].reset_index(drop=True),
    'Rolling Avg': rolling_avg_returns,
    'Rolling Beta': rolling_beta,
    'Rolling Std': rolling_std
})


We remove missing (NaN) values.

In [ ]:
rolling_df.dropna(inplace=True)

We test whether the relationship between beta coefficients and returns is statistically significant.

In [ ]:
rolling_avg_returns = rolling_df['Rolling Avg'].apply(lambda x: x.real)

y = rolling_avg_returns
X = rolling_df['Rolling Beta']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

We test whether the relationship between standard deviation and returns is statistically significant.

In [ ]:
rolling_avg_returns = rolling_df['Rolling Avg'].apply(lambda x: x.real)

y = rolling_avg_returns
X = rolling_df[['Rolling Std', 'Rolling Beta']]

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

Application of cross-sectional regression.

In [ ]:
subperiod_size = 29
num_subperiods = 10

regression_results_df = pd.DataFrame(columns=['Subperiod', 'Date From', 'Date To', 'Coef Beta', 'Pval Beta', 'Coef Std', 'Pval Std'])

for i in range(num_subperiods):
    start_index = i * subperiod_size
    end_index = start_index + subperiod_size
    subset = rolling_df.iloc[start_index:end_index]

    x_std = subset['Rolling Std']
    x_beta = subset['Rolling Beta']
    y = subset['Rolling Avg']

    x = pd.concat([x_std, x_beta], axis=1)
    x = sm.add_constant(x)
    model = sm.OLS(y, x)
    results = model.fit()

    coef_beta = results.params['Rolling Beta']
    pval_beta = results.pvalues['Rolling Beta']
    coef_std = results.params['Rolling Std']
    pval_std = results.pvalues['Rolling Std']

    subperiod_dates = subset.index
    date_from = subperiod_dates[0]
    date_to = subperiod_dates[-1]

    regression_results_df = regression_results_df.append({
        'Subperiod': i + 1,
        'Date From': date_from,
        'Date To': date_to,
        'Coef Beta': coef_beta,
        'Pval Beta': pval_beta,
        'Coef Std': coef_std,
        'Pval Std': pval_std
    }, ignore_index=True)

print(regression_results_df)


Calculation of correlation between generated returns and standard deviation indicators.

In [ ]:
corr, p_value = spearmanr(rolling_df1['Rolling Avg'], rolling_df1['Rolling Std'])
print('Spearman correlation: %.3f' % corr)
print('p-value: %.3f' % p_value)